In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
### Data Load 부분에서
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

# 개발을 위해 데이터 크기 줄이기 (예: 10%만 사용)
train = train.sample(frac=0.1, random_state=42)  # 전체 데이터의 10%만 사용
test = test.sample(frac=0.1, random_state=42)

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

In [ ]:
print(f"Training set shape: {train.shape}")
print(f"Test set shape: {test.shape}")
print("\nTarget distribution:")
print(y.value_counts(normalize=True))

In [ ]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [ ]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [ ]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [ ]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [ ]:
for col in numeric_columns:
    median_val = X_train_encoded[col].median()
    X_train_encoded[col] = X_train_encoded[col].fillna(median_val)
    X_test_encoded[col] = X_test_encoded[col].fillna(median_val)

In [ ]:
X_train_encoded['배아_성공률'] = X_train_encoded['미세주입에서 생성된 배아 수'] / (X_train_encoded['미세주입된 난자 수'] + 1e-6)
X_test_encoded['배아_성공률'] = X_test_encoded['미세주입에서 생성된 배아 수'] / (X_test_encoded['미세주입된 난자 수'] + 1e-6)

In [ ]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
scores = []


In [ ]:
model = ExtraTreesClassifier(
    n_estimators=100,  # 500 → 100으로 감소
    max_depth=15,
    min_samples_split=10,
    random_state=42
)

In [ ]:
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_encoded, y), 1):
    X_fold_train, X_fold_val = X_train_encoded.iloc[train_idx], X_train_encoded.iloc[val_idx]
    y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model.fit(X_fold_train, y_fold_train)
    pred = model.predict_proba(X_fold_val)[:, 1]
    score = roc_auc_score(y_fold_val, pred)
    scores.append(score)
    print(f"Fold {fold} ROC-AUC: {score:.4f}")

print(f"\nMean ROC-AUC: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")

In [ ]:
# 특성 중요도 시각화
feature_importance = pd.DataFrame({
    'feature': X_train_encoded.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(12, 6))
sns.barplot(data=feature_importance.head(20), x='importance', y='feature')
plt.title('Top 20 Most Important Features')
plt.tight_layout()
plt.show()

In [ ]:
model.fit(X_train_encoded, y)
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba
sample_submission.to_csv('./forsubmission.csv', index=False)